In [1]:
from math import radians, cos, sin, asin, sqrt

EARTH_RADIUS = 6_371 

def haversine_distance(lat1 : float , lon1 : float, lat2 : float, lon2 : float, unit : str = "m") -> float:
    R = EARTH_RADIUS if unit == "km" else EARTH_RADIUS * 1_000
    dLat = radians(lat2 - lat1)
    dLon = radians(lon2 - lon1)
    lat1 = radians(lat1)
    lat2 = radians(lat2)
    a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
    c = 2*asin(sqrt(a))
    return R * c

In [2]:
import json

STATUS = {
    "FREE": False,
    "DELIVERED": True,
    "TAKEN": True
}

class DeliveryOrder:
    def __init__(self, id : str, origin_lat : float, origin_long : float, dest_lat : float, dest_long : float, weight : int) -> None:
        self.id : str = id
        self.weight : int = weight
        
        self.start_position : dict = {
            "latitude": origin_lat,
            "longitude": origin_long
        }
        self.destination_position : dict = {
            "latitude": dest_lat,
            "longitude": dest_long
        }
        
        self.order_status : bool = STATUS["FREE"]
    
    def __str__(self) -> str:
        return "Order {} - from {} to {} with weight {}"\
            .format(self.id, (self.start_position['latitude'], 
                              self.start_position['longitude']), 
                            (self.destination_position['latitude'], 
                            self.destination_position['longitude']), 
                    self.weight)        
            
    def __repr__(self) -> str:
        return json.dumps({
            "id": self.id,
            "origin_lat": self.start_position['latitude'],
            "origin_long": self.start_position['longitude'],
            "dest_lat": self.destination_position['latitude'],
            "dest_long": self.destination_position['longitude'],
            "weight": self.weight
        })

In [3]:
import time

def timeit(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        for _ in range(1000):
            func(*args, **kwargs)
        end_time = time.time()
        print(f"Function '{func.__name__}' took {(end_time - start_time):.6f} seconds to run 1000 times.")
    return wrapper

In [12]:
# ----------------------------------------------------------------------------------------------

from itertools import combinations

# ---------------------------------------------------------------------------------------------

def closest_order(latitude, longitude, orders : list[DeliveryOrder]) -> DeliveryOrder:
    min_dist = float('inf')
    closest = None
    for order in orders:
        dist = haversine_distance(
            latitude,
            longitude,
            order.destination_position['latitude'],
            order.destination_position['longitude']
        )
        if dist < min_dist:
            min_dist = dist
            closest = order
    return closest

# ---------------------------------------------------------------------------------------------

def generate_path(orders: list[DeliveryOrder], first_order: DeliveryOrder) -> list[DeliveryOrder]:
    if not orders:
        return []
    start_order = first_order
    if start_order not in orders:
        return []
    path = [start_order]
    visited = {start_order.id} 
    current_order = start_order
    while len(path) < len(orders):
        next_order = None
        min_distance = float('inf')
        for order in orders:
            if order.id not in visited:  
                distance = haversine_distance(
                    current_order.destination_position['latitude'], 
                    current_order.destination_position['longitude'], 
                    order.destination_position['latitude'], 
                    order.destination_position['longitude']
                )
                if distance < min_distance:
                    min_distance = distance
                    next_order = order
        if next_order:
            visited.add(next_order.id)  
            path.append(next_order)
            current_order = next_order
        else:
            break 
    return path

# ---------------------------------------------------------------------------------------------

def calculate_travel_distance(path : list[DeliveryOrder]) -> float:
    if len(path) < 2:
        return 0.0
    total_distance = 0.0
    for i in range(len(path) - 1):
        start = path[i]
        end = path[i + 1]
        total_distance += haversine_distance(
            start.destination_position['latitude'], 
            start.destination_position['longitude'], 
            end.destination_position['latitude'], 
            end.destination_position['longitude']
        )
    return total_distance

# ---------------------------------------------------------------------------------------------

def calculate_capacity_level(orders: list[DeliveryOrder], max_capacity: int) -> float:
    total_weight = sum(order.weight for order in orders)
    capacity_level = total_weight / max_capacity
    return min(capacity_level, 1.0)

# ---------------------------------------------------------------------------------------------

def utility(num_orders: int, travel_distance: float, autonomy: float, capacity_level: float) -> float:
    if num_orders == 0:
        return float('-inf')
    capacity_utility = capacity_level
    if travel_distance > autonomy:
        travel_utility = float('-inf')
    else:
        travel_utility = 1 - (travel_distance / autonomy)
    final_utility = capacity_utility + travel_utility
    return final_utility

# ---------------------------------------------------------------------------------------------

def combine_orders(orders: list[DeliveryOrder], capacity: int) -> list[list[DeliveryOrder]]:
    valid_combinations = []
    for r in range(1, len(orders) + 1):
        valid_combinations.extend(
            list(combo) for combo in combinations(orders, r) if sum(order.weight for order in combo) <= capacity
        )
    return valid_combinations

# ---------------------------------------------------------------------------------------------

def best_available_orders(orders: list[DeliveryOrder], latitude: float, longitude: float, capacity: int, autonomy: float) -> list[DeliveryOrder]:
    order_sets = combine_orders(orders, capacity)
    best_set = None
    best_utility = float('-inf')
    for order_set in order_sets:
        closest = closest_order(latitude, longitude, order_set)
        distance_closest_order = haversine_distance(
            latitude, 
            longitude, 
            closest.destination_position['latitude'], 
            closest.destination_position['longitude']
        )
        path = generate_path(order_set, closest)
        travel_distance = distance_closest_order + calculate_travel_distance(path)
        capacity_level = calculate_capacity_level(order_set, capacity)
        set_utility = utility(len(order_set), travel_distance, autonomy, capacity_level)
        if set_utility >= best_utility:
            best_set = order_set
            best_utility = set_utility
    return best_set

# ----------------------------------------------------------------------------------------------

def best_order_decision(next_orders, position, max_capacity, autonomy, available_order_sets, warehouse_positions) -> str:
    winner = None
    drone_utility = float('-inf')
    
    if next_orders:
        orders = next_orders
        closest = closest_order(position["latitude"], position["longitude"], orders)
        distance_closest_order = haversine_distance(
            position["latitude"], 
            position["longitude"], 
            closest.destination_position['latitude'], 
            closest.destination_position['longitude']
        )
        path = generate_path(orders, closest)
        travel_distance = distance_closest_order + calculate_travel_distance(path)
            
        capacity_level = calculate_capacity_level(orders, max_capacity)
        drone_utility = utility(len(orders), travel_distance, autonomy, capacity_level)
    
    for warehouse, orders in available_order_sets.items():
        if next_orders:
            orders += next_orders
        distance_warehouse = haversine_distance(
            position["latitude"], 
            position["longitude"], 
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude']
        )
        closest_to_warehouse = closest_order(
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude'], 
            orders
        )
        distance_warehouse_to_closest_order = haversine_distance(
            warehouse_positions[warehouse]['latitude'], 
            warehouse_positions[warehouse]['longitude'], 
            closest_to_warehouse.destination_position['latitude'], 
            closest_to_warehouse.destination_position['longitude']
        )
        path = generate_path(orders, closest_to_warehouse)
        travel_distance = distance_warehouse + distance_warehouse_to_closest_order + calculate_travel_distance(path)
            
        orders += next_orders
        capacity_level = calculate_capacity_level(orders, max_capacity)
        new_utility = utility(len(orders), travel_distance, autonomy, capacity_level)
            
        if new_utility >= drone_utility:
            winner = warehouse
            drone_utility = new_utility  
    
    return winner

In [4]:
orders_w1 : list[DeliveryOrder] = [
    DeliveryOrder("w1_1", 18.994237, 72.825553, 19.017584, 72.922585, 20),
    DeliveryOrder("w1_2", 18.994237, 72.825553, 19.007584, 72.912585, 5),
    DeliveryOrder("w1_3", 18.994237, 72.825553, 19.007584, 72.912585, 15),
    DeliveryOrder("w1_4", 18.994237, 72.825553, 18.977584, 72.882585, 5),
    DeliveryOrder("w1_5", 18.994237, 72.825553, 19.017584, 72.922585, 15),
    DeliveryOrder("w1_6", 18.994237, 72.825553, 19.057584, 72.962585, 15),
    DeliveryOrder("w1_7", 18.994237, 72.825553, 18.957584, 72.862585, 20),
    DeliveryOrder("w1_8", 18.994237, 72.825553, 18.997584, 72.902585, 10),
    DeliveryOrder("w1_9", 18.994237, 72.825553, 19.057584, 72.962585, 10),
    DeliveryOrder("w1_10",18.994237, 72.825553, 18.937584, 72.842585, 10)
]

orders_w2 : list[DeliveryOrder] = [
    DeliveryOrder("w2_1", 18.927584, 72.832585, 19.037584, 72.942585, 5),
    DeliveryOrder("w2_2", 18.927584, 72.832585, 19.037584, 72.942585, 5),
    DeliveryOrder("w2_3", 18.927584, 72.832585, 18.947584, 72.852585, 5),
    DeliveryOrder("w2_4", 18.927584, 72.832585, 18.947584, 72.852585, 5),
    DeliveryOrder("w2_5", 18.927584, 72.832585, 18.977584, 72.882585, 20),
    DeliveryOrder("w2_6", 18.927584, 72.832585, 19.057584, 72.962585, 20),
    DeliveryOrder("w2_7", 18.927584, 72.832585, 18.957584, 72.862585, 10),
    DeliveryOrder("w2_8", 18.927584, 72.832585, 18.967584, 72.872585, 5),
    DeliveryOrder("w2_9", 18.927584, 72.832585, 19.017584, 72.922585, 15),
    DeliveryOrder("w2_10",18.927584, 72.832585, 18.937584, 72.842585, 10),
]


In [13]:
w1_position = {
    "latitude": 18.994237, 
    "longitude": 72.825553
}
best_w1_orders = best_available_orders(orders_w1, w1_position['latitude'], w1_position['longitude'], 20, 200)
print([order for order in best_w1_orders])

['w1_1'] 10526.425869824223 -inf
['w1_2'] 9269.798626034511 -inf
['w1_3'] 9269.798626034511 -inf
['w1_4'] 6276.065026615798 -inf
['w1_5'] 10526.425869824223 -inf
['w1_6'] 16034.846128957206 -inf
['w1_7'] 5636.835308220503 -inf
['w1_8'] 8107.648665074739 -inf
['w1_9'] 16034.846128957206 -inf
['w1_10'] 6549.192113912366 -inf
['w1_2', 'w1_3'] 9269.798626034511 -inf
['w1_2', 'w1_4'] 10867.05096075014 -inf
['w1_2', 'w1_5'] 10800.040452017578 -inf
['w1_2', 'w1_6'] 16920.573035347676 -inf
['w1_2', 'w1_8'] 9637.933916094702 -inf
['w1_2', 'w1_9'] 16920.573035347676 -inf
['w1_2', 'w1_10'] 17262.099480495555 -inf
['w1_3', 'w1_4'] 10867.05096075014 -inf
['w1_4', 'w1_5'] 12397.292769427773 -inf
['w1_4', 'w1_6'] 18517.82519677243 -inf
['w1_4', 'w1_8'] 9336.765718376986 -inf
['w1_4', 'w1_9'] 18517.82519677243 -inf
['w1_4', 'w1_10'] 12397.986579871947 -inf
['w1_8', 'w1_9'] 17288.70828205558 -inf
['w1_8', 'w1_10'] 15731.81428992057 -inf
['w1_9', 'w1_10'] 24912.87328411809 -inf
['w1_2', 'w1_4', 'w1_8'] 

In [14]:
w2_position = {
    "latitude": 18.927584, 
    "longitude": 72.832585
}
best_w2_orders = best_available_orders(orders_w2, w2_position['latitude'], w2_position['longitude'], 20, 40000)
print([order for order in best_w2_orders])

['w2_1'] 16834.090663220275 0.8291477334194931
['w2_2'] 16834.090663220275 0.8291477334194931
['w2_3'] 3061.1340136861536 1.1734716496578461
['w2_4'] 3061.1340136861536 1.1734716496578461
['w2_5'] 7652.510173140085 1.8086872456714977
['w2_6'] 19894.269594418027 1.5026432601395494
['w2_7'] 4591.636071833539 1.3852090982041614
['w2_8'] 6122.0947994924845 1.096947630012688
['w2_9'] 13773.737657079213 1.4056565585730196
['w2_10'] 1530.5886485689216 1.461735283785777
['w2_1', 'w2_2'] 16834.090663220275 1.079147733419493
['w2_1', 'w2_3'] 16834.090947883105 1.0791477263029223
['w2_1', 'w2_4'] 16834.090947883105 1.0791477263029223
['w2_1', 'w2_7'] 16834.09104290054 1.3291477239274865
['w2_1', 'w2_8'] 16834.09110633209 1.0791477223416979
['w2_1', 'w2_9'] 16834.09094856513 1.5791477262858717
['w2_1', 'w2_10'] 16834.090821312206 1.329147729467195
['w2_2', 'w2_3'] 16834.090947883105 1.0791477263029223
['w2_2', 'w2_4'] 16834.090947883105 1.0791477263029223
['w2_2', 'w2_7'] 16834.09104290054 1.32914

In [15]:
next_orders = []
max_capacity = 20
autonomy = 40000
available_order_sets = {
    "w1": best_w1_orders,
    "w2": best_w2_orders
}
warehouse_positions = {
    "w1": {
        "latitude": 18.994237, 
        "longitude": 72.825553
    },
    "w2": {
        "latitude": 18.927584, 
        "longitude": 72.832585
    }
}
position = {
    "latitude": warehouse_positions["w2"]["latitude"],
    "longitude": warehouse_positions["w2"]["longitude"]
}

winner = best_order_decision(next_orders, position, max_capacity, autonomy, available_order_sets, warehouse_positions)

print(f"Winner: {winner}")
available_order_sets[winner]

Winner: w2


[{"id": "w2_3", "origin_lat": 18.927584, "origin_long": 72.832585, "dest_lat": 18.947584, "dest_long": 72.852585, "weight": 5},
 {"id": "w2_4", "origin_lat": 18.927584, "origin_long": 72.832585, "dest_lat": 18.947584, "dest_long": 72.852585, "weight": 5},
 {"id": "w2_10", "origin_lat": 18.927584, "origin_long": 72.832585, "dest_lat": 18.937584, "dest_long": 72.842585, "weight": 10}]